<a href="https://colab.research.google.com/github/KhushiBhambri/YogNet/blob/main/PoseStreamModel/FinalPoseStream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

# import sys
# sys.path.append('/content/drive/MyDrive/Major_Project_Cse')

%cd "/content/drive/MyDrive/Colab_Notebooks"

!pip install import_ipynb
import import_ipynb
import GetKeypoints2 as GetKeypoints 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab_Notebooks
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.4 MB/s eta 0:00:00
importing Jupyter notebook from GetKeypoints2.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 47.9 MB/s eta 0:00:00


In [10]:
from google.colab import drive
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import initializers
from keras.layers import Conv3D, MaxPooling3D,Dense,ZeroPadding3D,Flatten,Activation,BatchNormalization,RandomFlip,RandomRotation,Rescaling
from keras.models import load_model



In [11]:
import pandas as pd

# read the data from the CSV file
data = pd.read_excel('train_data2.xlsx')

# load the keypoints and labels into separate variables
# train_keypoints = list(data['keypoints'])
train_keypoints=[]
train_labels = list(data['label'])

numOfKeypoints = 35
import ast
for keypoints_str in data['keypoints']:
    keypoints_list = ast.literal_eval(keypoints_str)
    train_keypoints.append(keypoints_list)

In [12]:
new_train_keypoints = []
new_train_labels = []
for i in range(len(train_keypoints)):
  keypoints = GetKeypoints.CentralisedKeypoints1(train_keypoints[i])
  if(keypoints is not None):
     new_train_keypoints.append(keypoints)
     new_train_labels.append(train_labels[i])


<string>:7: RuntimeWarning: Mean of empty slice
<string>:8: RuntimeWarning: Mean of empty slice


In [13]:
len(new_train_keypoints)

1213

In [14]:
len(new_train_keypoints[0][0])

35

In [16]:
# train_videos = np.asarray(train_videos)
import numpy as np
train_keypoints = np.asarray(train_keypoints)
train_labels = np.asarray(train_labels).astype('int32')

from sklearn.model_selection import train_test_split

XKey_train,XKey_test,yKey_train,yKey_test = train_test_split(train_keypoints,train_labels,test_size=0.25,shuffle=True)

In [18]:
import tensorflow as tf
XKey_train = tf.convert_to_tensor(XKey_train, dtype=tf.float32)
yKey_train = tf.convert_to_tensor(yKey_train, dtype=tf.int32)

In [19]:
from keras import models, layers
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, TimeDistributed, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.initializers import GlorotNormal
from keras.regularizers import L1L2

yKey_train = np.squeeze(yKey_train)

tf.config.run_functions_eagerly(True)

# Define model input shape
input_shape = (16, 35, 2)

# Build model architecture
model = models.Sequential()

model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu'),input_shape = input_shape))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=50, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))

# flatten output from CNN layers
model.add(TimeDistributed(GlobalAveragePooling1D()))

# add LSTM layer
# model.add(LSTM(100, dropout=0.0, recurrent_dropout=0.0, kernel_initializer=GlorotNormal(seed=42), recurrent_regularizer=L1L2(l1=0.0, l2=0.001), bias_regularizer=L1L2(l1=0.0, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))
model.add(LSTM(100, recurrent_regularizer=L1L2(l1=0.01, l2=0.001), bias_regularizer=L1L2(l1=0.01, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))

# TODO: forget bias 
# TODO: L1 = 0.01


# add fully connected layers

# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(80, activation='relu'))

# add Softmax layer
model.add(Dense(20, activation='softmax'))

# compile model
adam = Adam(learning_rate=0.001) 
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# print model summary
# print(model.summary())

history = model.fit(XKey_train,yKey_train,batch_size=32,epochs = 300,validation_split = 0.2)

Epoch 1/300


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


23/23 [==============================] - 16s 122ms/step - loss: 21.1285 - accuracy: 0.0509 - val_loss: 17.1137 - val_accuracy: 0.0824
Epoch 2/300
23/23 [==============================] - 2s 78ms/step - loss: 14.2138 - accuracy: 0.0798 - val_loss: 11.3942 - val_accuracy: 0.0989
Epoch 3/300
23/23 [==============================] - 2s 80ms/step - loss: 9.4259 - accuracy: 0.0990 - val_loss: 7.5754 - val_accuracy: 0.1044
Epoch 4/300
23/23 [==============================] - 2s 78ms/step - loss: 6.3116 - accuracy: 0.0963 - val_loss: 5.0763 - val_accuracy: 0.2088
Epoch 5/300
23/23 [==============================] - 2s 98ms/step - loss: 4.3053 - accuracy: 0.2215 - val_loss: 3.5396 - val_accuracy: 0.2308
Epoch 6/300
23/23 [==============================] - 2s 102ms/step - loss: 3.2863 - accuracy: 0.2765 - val_loss: 2.9862 - val_accuracy: 0.3022
Epoch 7/300
23/23 [==============================] - 2s 81ms/step - loss: 3.0030 - accuracy: 0.3040 - val_loss: 2.8390 - val_accuracy: 0.2967
Epoch 8/300

In [20]:
model.evaluate(XKey_test,yKey_test)

10/10 [==============================] - 0s 46ms/step - loss: 0.9317 - accuracy: 0.7796


[0.9316936135292053, 0.7796052694320679]

In [21]:
# Save the model to a file
model.save('PoseStream2.h5')